In [3]:
import polars as pl

commanders = pl.read_parquet("./../data/processed/commanders.parquet")
commanders.describe()

statistic,name,colorIdentity,colors,faceConvertedManaCost,keywords,loyalty,manaCost,manaValue,power,subtypes,supertypes,toughness,text,type,types,setCodes,page_content
str,str,str,str,f64,str,str,str,f64,str,str,str,str,str,str,str,f64,str
"""count""","""656""","""656""","""656""",35.0,"""461""","""0""","""647""",656.0,"""656""","""656""","""656""","""656""","""655""","""656""","""656""",656.0,"""656"""
"""null_count""","""0""","""0""","""0""",621.0,"""195""","""656""","""9""",0.0,"""0""","""0""","""0""","""0""","""1""","""0""","""0""",0.0,"""0"""
"""mean""",null,null,null,2.657143,null,null,null,3.920732,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,2.057166,null,null,null,1.520284,null,null,null,null,null,null,null,null,null
"""min""","""Abuelo, Ancest…","""""","""""",0.0,"""Adapt""",null,"""{10}""",1.0,"""*""","""Aetherborn, Ro…","""Legendary""","""*""","""({B/P} can be …","""Legendary Crea…","""Creature""",null,"""Legendary Crea…"
"""25%""",null,null,null,1.0,null,null,null,3.0,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,3.0,null,null,null,4.0,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,4.0,null,null,null,5.0,null,null,null,null,null,null,null,null,null
"""max""","""Éowyn, Shieldm…","""W""","""W""",7.0,"""Will of the co…",null,"""{X}{W}""",12.0,"""9""","""Zombie, Mutant…","""Legendary""","""9""","""{W/B}{W/B}: If…","""Legendary Crea…","""Creature""",null,"""Legendary Crea…"


In [4]:
commanders_subset = commanders.slice(0, 10)

In [5]:
import langchain
import langchain.globals
import langchain_openai

langchain.globals.set_debug(True)
langchain.globals.set_verbose(True)

llm = langchain_openai.OpenAI()

In [6]:

from langchain_community.document_loaders import PolarsDataFrameLoader

loader = PolarsDataFrameLoader(commanders_subset)

# Use lazy load for larger table, which won't read the full table into memory
for i in loader.lazy_load():
    print(i)

page_content="Flying, ward {2}\\n{1}{W}{U}: Exile another target creature or artifact you control. Return it to the battlefield under its owner's control at the beginning of the next end step." metadata={'name': 'Abuelo, Ancestral Echo', 'colorIdentity': 'U, W', 'colors': 'U, W', 'faceConvertedManaCost': None, 'keywords': 'Flying, Ward', 'loyalty': None, 'manaCost': '{1}{W}{U}', 'manaValue': 3.0, 'power': '2', 'subtypes': 'Spirit', 'supertypes': 'Legendary', 'toughness': '2', 'type': 'Legendary Creature — Spirit', 'types': 'Creature', 'setCodes': ['LCI', 'LCI'], 'page_content': "Legendary Creature — SpiritFlying, ward {2}\\n{1}{W}{U}: Exile another target creature or artifact you control. Return it to the battlefield under its owner's control at the beginning of the next end step.Flying, Ward"}
page_content="Nitro-9 — Whenever Ace, Fearless Rebel attacks, you may sacrifice an artifact. When you do, put a +1/+1 counter on Ace, Fearless Rebel, then it fights up to one target creature def

## Naive classification

In [11]:
from langchain.prompts import PromptTemplate

commander_classification = PromptTemplate.from_template(
    """Classify this commander: {commander_data}"""
)

prompts = []
for i in commanders_subset.rows():
    prompts.append(commander_classification.format(commander_data=i))

In [ ]:

responses = []
for i in prompts:
    llm.invoke(i)
    responses.append(llm.invoke(i))

print(responses)

## Few shot classification

In [8]:
example_commanders = commanders.filter(pl.col("name").is_in(["Atraxa, Grand Unifier", "Sauron, the Dark Lord", "The Necrobloom", "Gishath, Sun's Avatar", "Kumena, Tyrant of Orazca"]))

for i in example_commanders.rows():
    print(i)


('Atraxa, Grand Unifier', 'B, G, U, W', 'B, G, U, W', None, 'Deathtouch, Flying, Lifelink, Vigilance', None, '{3}{G}{W}{U}{B}', 7.0, '7', 'Phyrexian, Angel', 'Legendary', '7', 'Flying, vigilance, deathtouch, lifelink\\nWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)', 'Legendary Creature — Phyrexian Angel', 'Creature', ['ONE', 'ONE', 'ONE', 'ONE'], 'Legendary Creature — Phyrexian AngelFlying, vigilance, deathtouch, lifelink\\nWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, b

In [9]:
def commander_to_prompt_argument(df, commander_name) -> str:
    """Get single row that matches commnader name, then return a simple string with all formatting removed."""
    data = df.filter(pl.col("name") == commander_name).select(
        pl.first("name", "text", "power", "toughness", "manaCost", "type")
    )

    # for each value in each collumn append into a string
    data_str = ""
    for i in data.rows():
        for j in i:
            data_str += str(j) + " "

    # remove all \, \\n and {} from the string
    data_str = data_str.replace("\\", "").replace("\n", "").replace("{", "").replace("}", "")
    return data_str

commander_to_prompt_argument(example_commanders, "Atraxa, Grand Unifier")

'Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel '

In [12]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {
        "commander": commander_to_prompt_argument(example_commanders, "Atraxa, Grand Unifier"),
        "classification": "Types Matter, Voltron, Enters the battlefield",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Sauron, the Dark Lord"),
        "classification": "Ring Matters, Armies Aggro, Opponent Casts Benefit, Wheel",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders, "The Necrobloom"),
        "classification": "Lands Matter, Token Generation, Dredge",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Gishath, Sun's Avatar"),
        "classification": "Commander Damage, Dinosaur Tribal",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Kumena, Tyrant of Orazca"),
        "classification": "Merfolk Tribal, +1/+1 Counters",
    },
]

example_prompt = PromptTemplate(
    input_variables=["commander", "classification"],
    template="""Classify this commander: {commander}, Classifications: {classification}""",
)

print(example_prompt.format(**examples[0]))

Classify this commander: Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel , Classifications: Types Matter, Voltron, Enters the battlefield


In [13]:
query_commander = commander_to_prompt_argument(
    commanders, "Chiss-Goria, Forge Tyrant"
)

query_commander

'Chiss-Goria, Forge Tyrant Affinity for artifacts (This spell costs 1 less to cast for each artifact you control.)nFlying, hastenWhenever Chiss-Goria, Forge Tyrant attacks, exile the top five cards of your library. You may cast an artifact spell from among them this turn. If you do, it has affinity for artifacts. 5 4 6RRR Legendary Creature — Dragon '

In [14]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Commander: {commander}",
    input_variables=["commander"],
)

print(prompt.format(commander=query_commander))

Classify this commander: Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel , Classifications: Types Matter, Voltron, Enters the battlefield

Classify this commander: Sauron, the Dark Lord Ward—Sacrifice a legendary artifact or legendary creature.nWhenever an opponent casts a spell, amass Orcs 1.nWhenever an Army you control deals combat damage to a player, the Ring tempts you.nWhenever the Ring tempts you, you may discard your hand. If you do, draw four cards. 7 6 3UBR Legendary Creature — Avatar Horror , Classifications: Ring Matters, Armies Aggro, Opponent Casts Benefit, Wheel

C

In [ ]:
llm.invoke(prompt.format(commander=query_commander))

In [ ]:
llm.invoke(
    prompt.format(
        commander=commander_to_prompt_argument(commanders, "Cayth, Famed Mechanist")
    )
)

## Structured Output

In [15]:
commanders.filter(pl.col("name").is_in(["Cayth, Famed Mechanist"])).to_dict(as_series=False)

{'name': ['Cayth, Famed Mechanist'],
 'colorIdentity': ['R, U, W'],
 'colors': ['R, U, W'],
 'faceConvertedManaCost': [None],
 'keywords': ['Fabricate, Populate, Proliferate'],
 'loyalty': [None],
 'manaCost': ['{1}{U}{R}{W}'],
 'manaValue': [4.0],
 'power': ['3'],
 'subtypes': ['Dwarf, Artificer'],
 'supertypes': ['Legendary'],
 'toughness': ['3'],
 'text': ['Fabricate 1 (When this creature enters the battlefield, put a +1/+1 counter on it or create a 1/1 colorless Servo artifact creature token.)\\nOther nontoken creatures you control have fabricate 1.\\n{2}, {T}: Choose one —\\n• Populate.\\n• Proliferate.'],
 'type': ['Legendary Creature — Dwarf Artificer'],
 'types': ['Creature'],
 'setCodes': [['M3C', 'M3C', 'M3C', 'M3C', 'M3C']],
 'page_content': ['Legendary Creature — Dwarf ArtificerFabricate 1 (When this creature enters the battlefield, put a +1/+1 counter on it or create a 1/1 colorless Servo artifact creature token.)\\nOther nontoken creatures you control have fabricate 1.\\n

In [43]:
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field


class Commander(BaseModel):
    name: str = Field(title="Name of the commander", description="Name of the commander")
    colorIdentity: list[str] = Field(
        title="Color identity",
        description="The mana color identity of the commander when selecting cards for the deck",
    )
    manaCost: str = Field(
        title="Mana cost",
        description="The mana cost of the commander to pay to cast it",
    )
    type: str = Field(
        title="Type",
        description="The type of the commander",
    )
    text: str = Field(
        title="Text",
        description="The text of the commander that describes its abilities",
    )
    power: int = Field(
        title="Power",
        description="The power of the commander for combat",
    )
    toughness: int = Field(
        title="Toughness",
        description="The toughness of the commander for combat",
    )

def convert_commander(df: pl.DataFrame, name: str) -> Commander:
    if name not in df["name"]:
        raise ValueError(f"Commander {name} not found in DataFrame.")

    data = (
        df.filter(pl.col("name") == name)
        .select(
            pl.first(
                "name",
                "colorIdentity",
                "manaCost",
                "type",
                "text",
                "power",
                "toughness",
            )
        )
        .slice(0, 1)
        .to_dict(as_series=False)
    )

    def _strip_braces(s: str) -> str: return s.replace("{", "").replace("}", "")

    def _strip_new_line(s: str) -> str: return s.replace("\\n", " ")

    def _strip_backslash(s: str) -> str: return s.replace("\\", "")

    def _strip_all(s: str) -> str: return _strip_braces(_strip_backslash(_strip_new_line(s)))

    return Commander(
        name=data["name"][0],
        colorIdentity=data["colorIdentity"][0].split(", "),
        manaCost=_strip_all(data["manaCost"][0]),
        type=data["type"][0],
        text=_strip_all(data["text"][0]),
        power=int(data["power"][0]),
        toughness=int(data["toughness"][0]),
    )


convert_commander(commanders, "Adric, Mathematical Genius")

Commander(name='Adric, Mathematical Genius', colorIdentity=['U'], manaCost='1U', type='Legendary Creature — Human Artificer', text="2U, T: Copy target activated or triggered ability you control. You may choose new targets for the copy. Ultimate Sacrifice — 1U, Sacrifice Adric: Counter target activated or triggered ability. Doctor's companion (You can have two commanders if the other is the Doctor.)", power=1, toughness=1)

In [44]:
atraxa = convert_commander(commanders, "Atraxa, Grand Unifier")

str(atraxa)

"name='Atraxa, Grand Unifier' colorIdentity=['B', 'G', 'U', 'W'] manaCost='3GWUB' type='Legendary Creature — Phyrexian Angel' text='Flying, vigilance, deathtouch, lifelink When Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)' power=7 toughness=7"

In [50]:
class Classification(BaseModel):
    classification: list[str]


parser = PydanticOutputParser(pydantic_object=Classification)

atraxa_classification = Classification(
    classification=["Types Matter", "Voltron", "Enters the battlefield"]
)

str(atraxa_classification)

"classification=['Types Matter', 'Voltron', 'Enters the battlefield']"

In [42]:
class CommanderClassification(BaseModel):
    commander: Commander
    classification: Classification

atraxa_classified = CommanderClassification(commander=atraxa, classification=atraxa_classification)

atraxa_classified

CommanderClassification(commander=Commander(name='Atraxa, Grand Unifier', colorIdentity=['B', 'G', 'U', 'W'], manaCost='3GWUB', type='Legendary Creature — Phyrexian Angel', text='Flying, vigilance, deathtouch, lifelink When Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)', power=7, toughness=7), classification=Classification(classification=['Types Matter', 'Voltron', 'Enters the battlefield']))

In [47]:
commander_classification_query =convert_commander(commanders, "Adric, Mathematical Genius")

commander_classification_query

Commander(name='Adric, Mathematical Genius', colorIdentity=['U'], manaCost='1U', type='Legendary Creature — Human Artificer', text="2U, T: Copy target activated or triggered ability you control. You may choose new targets for the copy. Ultimate Sacrifice — 1U, Sacrifice Adric: Counter target activated or triggered ability. Doctor's companion (You can have two commanders if the other is the Doctor.)", power=1, toughness=1)

In [76]:
structured_examples = [
    {
        "commander": convert_commander(commanders, "Atraxa, Grand Unifier"),
        "classification": Classification(classification=["Types Matter", "Voltron", "Enters the battlefield"]),
    },
    {
        "commander": convert_commander(commanders, "Sauron, the Dark Lord"),
        "classification": Classification(
            classification=["Ring Matters", "Armies Aggro", "Opponent Casts Benefit", "Wheel"]
        ),
    },
    {
        "commander": convert_commander(commanders, "The Necrobloom"),
        "classification": Classification(
            classification=["Lands Matter", "Token Generation", "Dredge"]
        ),
    },
    {
        "commander": convert_commander(commanders, "Gishath, Sun's Avatar"),
        "classification": Classification(classification=["Commander Damage", "Dinosaur Tribal"]),
    },
    {
        "commander": convert_commander(commanders, "Kumena, Tyrant of Orazca"),
        "classification": Classification(classification=["Merfolk Tribal", "+1/+1 Counters"]),
    },
]

structured_examples

[{'commander': Commander(name='Atraxa, Grand Unifier', colorIdentity=['B', 'G', 'U', 'W'], manaCost='3GWUB', type='Legendary Creature — Phyrexian Angel', text='Flying, vigilance, deathtouch, lifelink When Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)', power=7, toughness=7),
  'classification': Classification(classification=['Types Matter', 'Voltron', 'Enters the battlefield'])},
 {'commander': Commander(name='Sauron, the Dark Lord', colorIdentity=['B', 'R', 'U'], manaCost='3UBR', type='Legendary Creature — Avatar Horror', text='Ward—Sacrifice a legendary artifact or legendary creature. Whenever an opponent casts a spell, amass Orcs 1. Whenever an Army you control deals combat damage to a 

In [77]:
structured_example_prompt = FewShotPromptTemplate(
    examples=structured_examples,
    example_prompt=PromptTemplate(
        input_variables=["commander", "classification"],
        template="""
        You are a Magic the Gathering expert who is assessing the possibilities for a new commander deck.

        Here are some examples of other commander cards and their classifications:
        Commander: {commander}, Classifications: {classification}""",
    ),
    suffix="Classify this commander: {commander}, Classification(classification=",
    input_variables=["commander"],
)

structured_example_prompt.format(**structured_examples[0])

'\n        You are a Magic the Gathering expert who is assessing the possibilities for a new commander deck.\n\n        Here are some examples of other commander cards and their classifications:\n        Commander: name=\'Atraxa, Grand Unifier\' colorIdentity=[\'B\', \'G\', \'U\', \'W\'] manaCost=\'3GWUB\' type=\'Legendary Creature — Phyrexian Angel\' text=\'Flying, vigilance, deathtouch, lifelink When Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)\' power=7 toughness=7, Classifications: classification=[\'Types Matter\', \'Voltron\', \'Enters the battlefield\']\n\n\n        You are a Magic the Gathering expert who is assessing the possibilities for a new commander deck.\n\n        Here are 

In [78]:
chain = structured_example_prompt | llm | parser

chain_invoke = chain.invoke(input={"commander": commander_classification_query})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [1:chain:RunnableSequence > 2:prompt:FewShotPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "You are a Magic the Gathering expert who is assessing the possibilities for a new commander deck.\n\n        Here are some examples of other commander cards and their classifications:\n        Commander: name='Atraxa, Grand Unifier' colorIdentity=['B', 'G', 'U', 'W'] manaCost='3GWUB' type='Legendary Creature — Phyrexian Angel' text='Flying, vigilance, deathtouch, lifelink When Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on 

OutputParserException: Invalid json output: ['Control', 'Activated Abilities', 'Triggered Abilities', 'Sacrifice', 'Doctor's companion'])